In [1]:
import cv2
import os
import numpy as np
from imutils import paths

In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [3]:
datapath= 'data'
outputmodel = r'New folder\VideoCMOdel'
outputLabelBInarizer = r'New folder\ModelBinarizer'


In [4]:
Sports_Label=['boxing', 'swimming','table_tennis','wwe' , 'hockey']
print("Image is Loaded....")
pathToImage = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImage:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Label:
        continue;
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (244,244))
    data.append(image)
    labels.append(label)
    
    
    


Image is Loaded....


In [5]:

labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [6]:
data = np.array(data)

In [7]:
(X_train, X_test,Y_train, Y_test) = train_test_split(data, labels, test_size = 0.25, stratify = labels, random_state =42)


In [8]:
traininAugmentation = ImageDataGenerator(

    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = "nearest"  
    
)


In [9]:
validationAugmentation = ImageDataGenerator()

mean = np.array([123.68, 116.779, 103.939], dtype="float32")
traininAugmentation.mean = mean
validationAugmentation.mean = mean


In [10]:
from keras.applications.resnet import ResNet50

from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras.layers.core import Dropout
from keras.models import Model

In [11]:
baseModel  = ResNet50(weights = 'imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name='Flatten')(headModel)
headModel = Dense(512, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs = baseModel.input, outputs=headModel)

for basemodeLayers in baseModel.layers:
    basemodeLayers.trainable = False

In [12]:
from keras.optimizers import SGD

In [13]:
epoch = 5

In [14]:
opt = SGD(lr=0.0001, momentum = 0.9, decay = 1e-4/epoch)

C:\Users\mk mobile\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [15]:
model.compile(loss= "categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [16]:
History = model.fit_generator(
    traininAugmentation.flow(X_train,Y_train, batch_size = 32),
    steps_per_epoch = len(X_train) // 32,
    validation_data = validationAugmentation.flow(X_test,Y_test),
    validation_steps = len(X_test) // 32,
    epochs = epoch
)

C:\Users\mk mobile\anaconda3\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
78/78 [==============================] - 423s 5s/step - loss: 2.3091 - accuracy: 0.2643 - val_loss: 0.9825 - val_accuracy: 0.6478
Epoch 2/5
78/78 [==============================] - 386s 5s/step - loss: 1.2587 - accuracy: 0.5155 - val_loss: 0.6808 - val_accuracy: 0.7740
Epoch 3/5
78/78 [==============================] - 382s 5s/step - loss: 0.9759 - accuracy: 0.6317 - val_loss: 0.5554 - val_accuracy: 0.8221
Epoch 4/5
78/78 [==============================] - 385s 5s/step - loss: 0.8358 - accuracy: 0.6803 - val_loss: 0.4875 - val_accuracy: 0.8377
Epoch 5/5
78/78 [==============================] - 405s 5s/step - loss: 0.7540 - accuracy: 0.7290 - val_loss: 0.4457 - val_accuracy: 0.8425


In [17]:
import pickle

In [18]:
model.save(outputmodel)
lbinarizer = open(r'New folder\ModelBinarizer\ModelBinarizer.pickle', "wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()

C:\Users\mk mobile\anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: New folder\VideoCMOdel\assets
